# Introduction to XGBoost-Spark Cross Validation with GPU

The goal of this notebook is to show you how to levarage GPU to accelerate XGBoost spark cross validatoin for hyperparameter tuning. The best model for the given hyperparameters will be returned.

Here takes the application 'Mortgage' as an example.

A few libraries are required for this notebook:
  1. NumPy
  2. cudf jar
  2. xgboost4j jar
  3. xgboost4j-spark jar

#### Import the Required Libraries

In [1]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType
from time import time

As shown above, here `CrossValidator` is imported from package `ml.dmlc.xgboost4j.scala.spark.rapids`, not the spark's `tuning.CrossValidator`.

#### Create a Spark Session

In [2]:
spark = SparkSession.builder.appName("mortgage-cv-gpu-python").getOrCreate()

#### Specify the Data Schema and Load the Data

In [3]:
label = 'delinquency_12'
schema = StructType([
    StructField('orig_channel', FloatType()),
    StructField('first_home_buyer', FloatType()),
    StructField('loan_purpose', FloatType()),
    StructField('property_type', FloatType()),
    StructField('occupancy_status', FloatType()),
    StructField('property_state', FloatType()),
    StructField('product_type', FloatType()),
    StructField('relocation_mortgage_indicator', FloatType()),
    StructField('seller_name', FloatType()),
    StructField('mod_flag', FloatType()),
    StructField('orig_interest_rate', FloatType()),
    StructField('orig_upb', IntegerType()),
    StructField('orig_loan_term', IntegerType()),
    StructField('orig_ltv', FloatType()),
    StructField('orig_cltv', FloatType()),
    StructField('num_borrowers', FloatType()),
    StructField('dti', FloatType()),
    StructField('borrower_credit_score', FloatType()),
    StructField('num_units', IntegerType()),
    StructField('zip', IntegerType()),
    StructField('mortgage_insurance_percent', FloatType()),
    StructField('current_loan_delinquency_status', IntegerType()),
    StructField('current_actual_upb', FloatType()),
    StructField('interest_rate', FloatType()),
    StructField('loan_age', FloatType()),
    StructField('msa', FloatType()),
    StructField('non_interest_bearing_upb', FloatType()),
    StructField(label, IntegerType()),
])
features = [ x.name for x in schema if x.name != label ]

train_data = spark.read.parquet('/data/mortgage/parquet/train')
trans_data = spark.read.parquet('/data/mortgage/parquet/eval')

#### Build a XGBoost-Spark CrossValidator

In [4]:
# First build a classifier of GPU version using *setFeaturesCols* to set feature columns
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 10, 
    'maxLeaves': 256,
    'growPolicy': 'depthwise',
    'objective': 'binary:logistic',
    'minChildWeight': 30.0,
    'lambda_': 1.0,
    'scalePosWeight': 2.0,
    'subsample': 1.0,
    'nthread': 1,
    'numRound': 100,
    'numWorkers': 1,
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)
# Then build the evaluator and the hyperparameters
evaluator = (MulticlassClassificationEvaluator()
    .setLabelCol(label))
param_grid = (ParamGridBuilder()
    .addGrid(classifier.maxDepth, [3, 6])
    .addGrid(classifier.numRound, [100, 200])
    .build())
# Finally the corss validator
cross_validator = (CrossValidator()
    .setEstimator(classifier)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(param_grid)
    .setNumFolds(3))

#### Start Cross Validation by Fitting Data to CrossValidator

In [5]:
def with_benchmark(phrase, action):
    start = time()
    result = action()
    end = time()
    print('{} takes {} seconds'.format(phrase, round(end - start, 2)))
    return result
model = with_benchmark('Cross-Validation', lambda: cross_validator.fit(train_data)).bestModel

Cross-Validation takes 88.53 seconds


#### Transform On the Best Model

In [7]:
def transform():
    result = model.transform(trans_data).cache()
    result.foreachPartition(lambda _: None)
    return result
result = with_benchmark('Transforming', transform)
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

Transforming takes 3.13 seconds
+--------------+--------------------+--------------------+----------+
|delinquency_12|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+----------+
|             0|[2.57163572311401...|[0.92901364713907...|       0.0|
|             0|[2.63977861404418...|[0.93337820470333...|       0.0|
|             0|[2.50156974792480...|[0.92425179481506...|       0.0|
|             0|[2.63977861404418...|[0.93337820470333...|       0.0|
|             0|[2.09173870086669...|[0.89009761810302...|       0.0|
+--------------+--------------------+--------------------+----------+
only showing top 5 rows



#### Evaluation

In [8]:
accuracy = with_benchmark(
    'Evaluation',
    lambda: MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result))
print('Accuracy is ' + str(accuracy))

Evaluation takes 0.29 seconds
Accuracy is 0.9868033296704449


In [9]:
spark.stop()